# Pfam Protein Sequence Classification with Tensorflow and Keras

Adapted from Saleh Alkhalifa. [Machine Learning in Biotechnology and Life Sciences](https://github.com/PacktPublishing/Machine-Learning-in-Biotechnology-and-Life-Sciences).

In this tutorial, we will attempt to develop a protein sequence classification model in which we will classify protein sequences based on their known family accession using the Pfam (https://pfam.xfam.org/) dataset.

Pfam: The protein families database in 2021: J. Mistry, S. Chuguransky, L. Williams, M. Qureshi, G.A. Salazar, E.L.L. Sonnhammer, S.C.E. Tosatto, L. Paladin, S. Raj, L.J. Richardson, R.D. Finn, A. Bateman
Nucleic Acids Research (2020) doi: 10.1093/nar/gkaa913

The Pfam dataset consists of several columns, as follows:

- *Family_id*: The name of the family that the seqeunce belongs to (for example, filamin).
- *Family Accession*: The class or output that our model will aim to predict.
- *Sequence*: The amino acid sequence we will use as input for our model

We will use the seqeunce data to develop model to determine each seqeunece's associated family accession. The sequences are in their raw state with different lengths and sizes. We will need to pre-process the data and structure it in such a way as to prepare it for sequence classification. We will develop a model using a *balanced* set of different labels to ensure the model does not learn any particular bias.

- **TensorFlow** is an end-to-end open source platform for machine learning. It has a comprehensive, flexible ecosystem of tools, libraries and community. 
- **Keras** is a deep learning API written in Python, running on top of the machine learning platform TensorFlow.



## Import libraries

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.layers import (
    LSTM,
    Bidirectional,
    Conv1D,
    Dense,
    Dropout,
    Embedding,
    Flatten,
    Input,
    MaxPooling1D,
)
from keras.models import Model, Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.regularizers import l2
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

sns.set_style("darkgrid")


## Download dataset

In [ ]:
URL = "https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-in-Biotechnology-and-Life-Sciences/main/datasets/dataset_pfam"

files = []
for i in range(8):
    df = pd.read_csv(f"{URL}/dataset_pfam_seq_sd{i+1}.csv", index_col=None, header=0)
    files.append(df)

df = pd.concat(files, axis=0, ignore_index=True)
df.shape

### Examine the data

Peek into the data.

In [ ]:
df.head()

Check missing data.

In [ ]:
df.isna().sum()

Get top 10 abundant family ids.

In [ ]:
df["family_id"].groupby(df["family_id"]).value_counts().nlargest(10)

Get top 10 abundant family accessions.

In [ ]:
df["family_accession"].groupby(df["family_accession"]).value_counts().nlargest(10)

Plot the sequence length frequency distribution.

In [ ]:
sns.displot(df["sequence"].apply(lambda x: len(x)), bins=75, height=4, aspect=2)

Get mean sequence length.

In [ ]:
df["sequence"].str.len().mean()

Get min sequence length.

In [ ]:
df["sequence"].str.len().min()

Get max sequence length.

In [ ]:
df["sequence"].str.len().max()

Get median sequence length.

In [ ]:
df["sequence"].str.len().median()

Get family accessions with counts more than 1200.

In [ ]:
df_filt = df.groupby("family_accession").filter(lambda x: len(x) > 1200)
df_filt

## Create a balanced dataset

In [ ]:
df_bal = df_filt.groupby("family_accession").apply(lambda x: x.sample(1200))
df_bal.family_accession.value_counts()

In [ ]:
# Peek into the balanced dataset
df_bal.head()

## Prepare input dataframe for modeling

`reset_index` in pandas is used to reset index of the dataframe object to default indexing (0 to number of rows minus 1) or to reset multi level index. By doing so, the original index gets converted to a column.

In [ ]:
df_red = df_bal[["family_accession", "sequence"]].reset_index(drop=True)
df_red.head()

Compute num of unique classes.

In [ ]:
num_classes = len(df_red.family_accession.value_counts())
num_classes

Get Pfam family accession unique number counts.

In [ ]:
df_red.family_accession.value_counts()

### Make train and test datasets

Split data into 75% X_train and 25% X_Test, among `X_Test`, 50% for validation (`X_val`) and 50% for test (`X_test`).

In [ ]:
X_train, X_Test = train_test_split(df_red, test_size=0.25)
X_val, X_test = train_test_split(X_Test, test_size=0.50)

Get the train, test, and validation dataset sizes.

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

### Create amino acid sequence dictionary

In [ ]:
aa_seq_dict = {
    "A": 1,
    "C": 2,
    "D": 3,
    "E": 4,
    "F": 5,
    "G": 6,
    "H": 7,
    "I": 8,
    "K": 9,
    "L": 10,
    "M": 11,
    "N": 12,
    "P": 13,
    "Q": 14,
    "R": 15,
    "S": 16,
    "T": 17,
    "V": 18,
    "W": 19,
    "Y": 20,
}

In [ ]:
# Encode amino acid sequence using the dictionary above
def aa_seq_encoder(data):
    full_sequence_list = []
    for i in data["sequence"].values:
        row_sequence_list = []
        for j in i:
            row_sequence_list.append(aa_seq_dict.get(j, 0))
        full_sequence_list.append(np.array(row_sequence_list))
    return full_sequence_list


X_train_encode = aa_seq_encoder(X_train)
X_val_encode = aa_seq_encoder(X_val)
X_test_encode = aa_seq_encoder(X_test)

In [ ]:
# Show an example encoded amino acid sequence
X_train_encode[0]

Pad sequence to the same length of 100

In [ ]:
max_length = 100

X_train_padded = pad_sequences(
    X_train_encode, maxlen=max_length, padding="post", truncating="post"
)
X_val_padded = pad_sequences(
    X_val_encode, maxlen=max_length, padding="post", truncating="post"
)
X_test_padded = pad_sequences(
    X_test_encode, maxlen=max_length, padding="post", truncating="post"
)

In [ ]:
X_train.sequence[1]

In [ ]:
X_train_encode[1][:]

In [ ]:
X_train_padded[1][:]

Encode target labels with value between `0` and `n_classes-1`

In [ ]:
le = LabelEncoder()

y_train_enc = le.fit_transform(X_train["family_accession"])
y_val_enc = le.transform(X_val["family_accession"])
y_test_enc = le.transform(X_test["family_accession"])

In [ ]:
X_train["family_accession"]

In [ ]:
y_train_enc

In [ ]:
num_classes = len(le.classes_)
num_classes

In [ ]:
# Converts a class vector (integers) to binary class matrix.

y_train = to_categorical(y_train_enc)
y_val = to_categorical(y_val_enc)
y_test = to_categorical(y_test_enc)

In [ ]:
y_train

## Build model

In [ ]:
# Sequential groups a linear stack of layers into a tf.keras.Model.
# Sequential provides training and inference features on this model.
model = Sequential()

# EmbeddingLayer: Turns positive integers (indexes) into dense vectors of fixed size.
#  input_dim: Integer. Size of the vocabulary, i.e. maximum integer index + 1.
#  output_dim: Integer. Dimension of the dense embedding.
#  input_length: Length of input sequences, when it is constant.
model.add(Embedding(21, 16, input_length=max_length, name="EmbeddingLayer"))

# Bidirectional wrapper for RNNs with 16 units of LSTM
model.add(Bidirectional(LSTM(16), name="BidirectionalLayer"))

# Applies Dropout to the input with 20% of the input units to drop.
model.add(Dropout(0.2, name="DropoutLayer"))

# densely-connected NN layer of 28 units
model.add(Dense(28, activation="softmax", name="DenseLayer"))

# Optimizer that implements the Adam algorithm
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

# Configures the model for training use 'Adam' as optimizer, 'categorical_crossentropy'
# as loss funciton, and 'accuracy' as evaluation metrics
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Stop training when a monitored metric has stopped improving

In [ ]:
# monitor: Quantity to be monitored.
# patience: Number of epochs with no improvement after which training will be stopped.
# verbose: verbosity mode on
es = EarlyStopping(monitor="val_loss", patience=5, verbose=1)

(The following cell may takeu a few minutes.)

In [ ]:
# Trains the model for a fixed number of epochs (iterations on a dataset)
history = model.fit(
    X_train_padded,
    y_train,
    epochs=100,
    batch_size=256,
    validation_data=(X_val_padded, y_val),
    callbacks=[es],
)

### Plot accuracy and loss

In [ ]:
# Plot accuracy and loss across epochs
fig = plt.figure(figsize=(10, 10))

# total_rows, total_columns, subplot_index(1st, 2nd, etc..)
plt.subplot(2, 2, 1)
plt.title("Accuracy", fontsize=15)
plt.xlabel("Epochs", fontsize=15)
plt.ylabel("Accuracy (%)", fontsize=15)
plt.plot(
    history.history["val_accuracy"], label="Validation Accuracy", linestyle="dashed"
)
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.legend(["Validation", "Training"], loc="lower right")

plt.subplot(2, 2, 2)
plt.title("Loss", fontsize=15)
plt.xlabel("Epochs", fontsize=15)
plt.ylabel("Loss", fontsize=15)
plt.plot(history.history["val_loss"], label="Validation loss", linestyle="dashed")
plt.plot(history.history["loss"], label="Training loss")
plt.legend(["Validation", "Training"], loc="upper right")

## Generate predictions

In [ ]:
# Generates output predictions for the input samples
y_pred = model.predict(X_test_padded)

# Build a text report showing the main classification metrics
print(
    classification_report(
        np.argmax(y_test, axis=1),
        np.argmax(y_pred, axis=1),
        target_names=le.classes_,
    )
)

- **Support** is the number of actual occurrences of the class in the specified dataset. 
- **Macro avg** takes the arithmetic mean (aka unweighted mean). 
- **Weighted avg** takes the mean of all per-class while considering each class’s support.

### Show predicted values

In [ ]:
y_pred

### Confusion matrix

Compute confusion matrix to evaluate the accuracy of a classification.

In [ ]:
cf_matrix = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))

Plot confusion_matrix

In [ ]:
plt.figure(figsize=(15, 10))
sns.heatmap(cf_matrix, annot=True, fmt="", cmap="Blues")